In [1]:
import pandas as pd
import numpy as np
import json
from sqlalchemy.engine import URL
from sqlalchemy import create_engine

credenciales_path = r"C:\Users\fcolin\Desktop\input\Credenciales.json"
# Credenciales de SQL Server
with open(credenciales_path) as f:
    credenciales = json.load(f)
  # Credenciales de SQL Server
with open(credenciales_path) as f:
    credenciales = json.load(f)

connection_string = 'DRIVER={SQL Server};SERVER=Shjet-prod;DATABASE=Allocations;UID='+ credenciales['usuario'] +';PWD='+credenciales['password']
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
engine = create_engine(connection_url)
conn = engine.connect()

query_items = '''
SELECT DISTINCT ---TOP(10000)
[Original Vendor Item No_], [No_], [Description], [Division Code], [Item Category Code],[Product Group Code], [Unit Cost]
FROM   [Allocations].[dbo].[Item_BC_v2] 
WHERE LEN([Original Vendor Item No_]) = 7
---ORDER BY [Original Vendor Item No_] ---DESC

    '''
    
df_item = pd.read_sql_query(query_items, conn)

In [2]:
#df_items = df_items[['Original Vendor Item No_', 'No_', 'Variant Code', 'Description', 'Division Code', 'Item Category Code','Product Group Code']]
# Incluimos unit cost, con el podemos comparar si la predccion de descripciones coinciden o se acercan al precio como método de validación
df_items = df_item[['Original Vendor Item No_', 'No_', 'Description', 'Division Code', 'Item Category Code','Product Group Code', 'Unit Cost']]
# seleccionar los rows que en orignial vendor item no_  contengan solo valores numericos
df_items = df_items[df_items['Original Vendor Item No_'].str.isnumeric()]
# Aqui vamos a homologar los datos, se supone que la columna No_ solo se usa si existe algun clon de un item, si orginal y No_ son iguales, no tiene clon 
# entonces si No_ es igual a Original Vendor Item No_ entonces se elimina elimna el valor de No_ y se deja vacio
df_items['No_'] = np.where(df_items['No_'] == df_items['Original Vendor Item No_'], '', df_items['No_'])

In [3]:
# Ahora, existen casos en los que ya existe un item padre y dan de alta el clon como padre
# detectar si alguno de los valores en la columna No_ Se encuentran en la columna Original Vendor Item No_
if df_items['No_'].isin(df_items['Original Vendor Item No_']).any():
    # valor que se encuentra en la columna No_ que se encuentra en la columna Original Vendor Item No_
    no_ligado =  df_items.loc[df_items['No_'].isin(df_items['Original Vendor Item No_']), ('Original Vendor Item No_', 'No_')].reset_index(drop=True) 
    for i in no_ligado.index:
        padre = no_ligado.loc[i, 'Original Vendor Item No_']
        clon = no_ligado.loc[i, 'No_']
        # Entonces, si en el original vendor hay un código de un clon, este se sustituye por el padre de dicho clon
        df_items.loc[df_items['Original Vendor Item No_'] == clon, 'Original Vendor Item No_'] = padre

    print(no_ligado)
        

    Original Vendor Item No_      No_
0                    1404930  1500818
1                    1415320  1500816
2                    1416322  1801246
3                    1500965  1501431
4                    1501860  1503953
..                       ...      ...
145                  2200911  2206872
146                  2201100  2113124
147                  2201125  2207540
148                  2201126  2208510
149                  2210605  2215341

[150 rows x 2 columns]


In [4]:
# Eliminamos los carácteres que estan entre parentesis y los guardamos en una nueva columna
df_items['parenthesis'] = ''
for i in df_items.index:
    if df_items.loc[i,'Description'].find('(') != -1 or df_items.loc[i,'Description'].find(')') != -1:
        sp = df_items.loc[i,'Description'].find('(') # sp = start parenthesis
        pp = df_items.loc[i,'Description'].find(')') # pp = end parenthesis
        inic = df_items.loc[i,'Description'][sp+1:pp]
        df_items.loc[i,'parenthesis']  = inic
        df_items.loc[i,'Description'] = df_items.loc[i,'Description'].replace('('+inic+')','')
        

In [5]:
df_items['Description'] = df_items[('Description')].str.replace(r"\(.*\)","")
# Eliminando acentos y caracteres especiales
df_items['Description'] =  df_items['Description'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')


C:\Users\fcolin\AppData\Local\Temp\ipykernel_17032\2468950121.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_items['Description'] = df_items[('Description')].str.replace(r"\(.*\)","")


In [6]:
# Esta bien que se el,iné? o deberíamos mejor eliminar este tipo de carácteres? 
df_items['Description'] = df_items['Description'].str.replace('C/', 'CON ')

In [7]:
# espacio en blanco al iniciar la descripcion por quitar los parentesis, se elimnia si existe
df_items['Description'] = df_items['Description'].str.strip()
# eliminando espacios entre palabras
df_items['Description'] = df_items['Description'].str.replace(' +', ' ')
df_items = df_items.drop_duplicates()
# No quiero hacer mas limpieza para no alterar la muestra


C:\Users\fcolin\AppData\Local\Temp\ipykernel_17032\4223025019.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df_items['Description'] = df_items['Description'].str.replace(' +', ' ')


In [8]:
# Primier Análisis 
# Existen prductos que incluso aunque esten ligados tienen diferentes descripciones, con ellos podría realizarse el primer entrenamiento?
productos = df_items['Original Vendor Item No_'].unique()

item_dif_descripcion = []
for producto in productos:
 df = df_items[df_items['Original Vendor Item No_'] == producto]
 if len(df['Description'].unique()) > 1:
     item_dif_descripcion.append(producto)
     print(producto)
     print(df['Description'].unique())
     print('')

1101208
['SANDALIA CRUZADA' 'SANDALIA BASICA CRUZADA']

1103091
['BOTA LISA FRUNCE' 'BOTA BASICA']

1204972
['PIERCING NARIZ BRILLO' 'PIERCING PIEDRA B120']

1303187
['SANDALIA BASICA NUDO' 'SANDALIA NUDO CANVAS']

1304012
['STRAPLESS TUBE BASIC TOP' 'BLUSA STRAPLESS TUBO']

1404930
['FALDA MIDI RAYAS' 'FALDA MIDI SOLIDO']

1412391
['M/3/4 CARDIGAN BOTONES' 'CARDIGAN M/3/4 BOTONES']

1412607
['10 PIEZAS LIGA BASICA' 'LIGA COLORES ESCOLAR' '12PCS LIGA ESCOLAR'
 '16 PZ LIGA ESCOLAR' 'SET LIGA ESCOLAR' 'LIGA BASICA MULTICOLOR ESCOLAR']

1414827
['SACO CROP MANGA 3/4' 'SACO CROP MANGA 3/4_x000D_\n_x000D_']

1415320
['VESTIDO MAXI RAYAS 2FER' 'VESTIDO MAXI SOLIDO' 'VESTIDO MAXI FLORAL'
 'VESTIDO MAXI CACHEMIR']

1416322
['CARDIGAN BASICO BOTONES' 'CARDIGAN M/L']

1416462
['VESTIDO SKATER RECRTE M3/4' 'VESTIDO SKATER M3/4']

1500478
['BALSAMO LABIAL BOLITA' 'BALSAMO LABIAL BOLITA CHA']

1500965
['VESTIDO MAXI S/T ESTAMPADO' 'VESTIDO MAXI S/T FLORES']

1501019
['BLUSA CON CIERRE M3/4' 'BLUA C

In [9]:
# Aquí podríamos hacer un análisis léxico, para ver si hay palabras que no aportan valor, como por ejemplo
'''palabras = ''.join(df_items['Description'].unique())
palabras = palabras.split(' ')
#palabras = list(set(palabras))
palabras.sort()
print(palabras)
# Eliminando palabras que no aportan valor
#print([palabra for palabra in palabras if len(palabra) <= 2])'''

"palabras = ''.join(df_items['Description'].unique())\npalabras = palabras.split(' ')\n#palabras = list(set(palabras))\npalabras.sort()\nprint(palabras)\n# Eliminando palabras que no aportan valor\n#print([palabra for palabra in palabras if len(palabra) <= 2])"

In [10]:
# Existirá el caso en el que un producto este ligado pero sean productos diferentes? Tomar a condsideracion despues
# Muestra representativa? 
len(item_dif_descripcion)

700

In [11]:
# suffle para que no se vea el orden de los items
import random
random.shuffle(item_dif_descripcion)

In [12]:
train = item_dif_descripcion[:int(len(item_dif_descripcion)*0.8)] # 80,20
test = item_dif_descripcion[int(len(item_dif_descripcion)*0.8):]

In [13]:
test_set = df_items[df_items['Original Vendor Item No_'].isin(test)]
train_set = df_items[df_items['Original Vendor Item No_'].isin(train)]
train_set

,Original Vendor Item No_,No_,Description,Division Code,Item Category Code,Product Group Code,Unit Cost,parenthesis
70,1101208,,SANDALIA CRUZADA,W-SHO,SANDALS,WEDGE S,95.000,ESSE
71,1101208,1503724,SANDALIA BASICA CRUZADA,W-SHO,SANDALS,CASUAL S,75.000,
83,1103091,,BOTA LISA FRUNCE,W-SHO,BOOTS,UNDER K BT,169.950,
84,1103091,1201937,BOTA BASICA,W-SHO,BOOTS,UNDER K BT,169.920,
229,1204972,,PIERCING NARIZ BRILLO,W-JEW,EARRINGS,STUDS,8.045,
...,...,...,...,...,...,...,...,...
86075,2209282,2211562,ARETES 2 GOTAS COLOR,W-JEW,EARRINGS,DROPS,14.010,
86504,2209719,,SANDALIA DOS TIRAS EVA SLIDE,W-SHO,SANDALS,CASUAL S,112.000,
86505,2209719,2215278,SANDALIA DOS TIRAS EVA SLIDES,W-SHO,SANDALS,CASUAL S,85.000,
90310,2213681,,GORRA MASCOTA CAMUFLAGE,W-ACC,PET,PET WEAR,72.500,


In [14]:
training_labels = train_set['Original Vendor Item No_'].values
training_sentences = train_set['Description'].values

testing_labels = test_set['Original Vendor Item No_'].values
testing_sentences = test_set['Description'].values


In [15]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

vocab_size = 10000
max_length = 120 # defalut
embedding_dim = 16 # default
trunc_type='post'
oov_tok = "<OOV>"

# Tokenizer
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)

# Generamos el diccionario de palabras para las oraciones de entrenamiento
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

# Genereamos tokens para las oraciones de entrenamiento y de prueba
sequences = tokenizer.texts_to_sequences(training_sentences)
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)

# Padding: matriz de ceros para que todas las oraciones tengan el mismo tamaño
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length, truncating=trunc_type)

# vale hasta aquí el ya tenemos la tokenización de las oraciones y el padding para que todas tengan el mismo tamaño
# ahora usaremos un modelo de clasificación? 
# para que el orden de las oraciones tenga sentido podemos itulizar un LSTM
# un embedding layer para que las palabras se representen en un espacio de det dimensión
# un dense layer para la clasificación

# Ya he hecho un modelo para clasificacion de sentimientos, pero no se si es el adecuado para este caso
# se clasificaba en 5 labaeles, podríamos hacer esto para clasificar con el sku padre y compararlo con el Unit Cost para combrobar 
# pero ahora no se si aguante ese tipo de clasificación pues existen muchos items entonces serían muchas clasificaciones
# ideas?

In [17]:
testing_padded # correlaciones? 

array([[  0,   0,   0, ...,  12, 154,   9],
       [  0,   0,   0, ..., 100,   9,   1],
       [  0,   0,   0, ...,   1,   1,   1],
       ...,
       [  0,   0,   0, ..., 162,   1,   1],
       [  0,   0,   0, ..., 526,   1, 464],
       [  0,   0,   0, ..., 150,   1, 464]])

In [ ]:
# Sugerencia de red, ajustar a nuestro caso y a nuestra clasificaciones
''' 
training_padded = np.array(padded)
training_labels = np.array(training_labels) - 1

import tensorflow as tf

# Build the model
model = tf.keras.Sequential([ 
        # This is how you need to set the Embedding layer when using pre-trained embeddings
        tf.keras.layers.Embedding(vocab_size+1, embedding_dim, input_length=maxlen, weights=[embeddings_matrix], trainable=False), 
#         tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=maxlen),
        tf.keras.layers.Dropout(0.2),
#         tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
        tf.keras.layers.Conv1D(32, 5, activation='relu'),
        tf.keras.layers.GlobalMaxPooling1D(),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    
model.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy']) 
# Print the model summary
model.summary()

'''